In [10]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
SERVICE_ACCOUNT_FILE = "key.json"

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)

SHEET_ID = "1KaB4ti8NissgX45bOuRpnoE07qknv7VQk_ECJ1zpJt0"
RANGE = "Star Wars Universe!A2:E155"

service = build("sheets", "v4", credentials=credentials)
sheet = service.spreadsheets()
result = (
    sheet.values()
    .get(
        spreadsheetId=SHEET_ID,
        range=RANGE,
        # valueRenderOption="FORMULA",
    )
    .execute()
)
values = result['values']

In [3]:
def parse_date(date_str):
    if date_str == '':
        return
    parts = date_str.split('-')
    parts = [p.strip() for p in parts]
    if len(parts) == 1:
        [year, label] = parts[0].split(' ')
        year = float(year)
        if label == 'BBY':
            year *= -1 
        return [year]
    elif len(parts) == 2:
        [p1, p2] = parts
        p1_parts = p1.split(' ')
        y1 = float(p1_parts[0])
        if len(p1_parts) == 2 and p1_parts[1] == 'BBY':
            y1 *= -1
        p2_parts = p2.split(' ')
        y2 = float(p2_parts[0])
        if p2_parts[1] == 'BBY':
            y2 *= -1
            if y1 > 0:
                y1 *= -1
        return [y1, y2]
    return None

def parse_rdate(date_str):
    if date_str == '':
        return
    parts = date_str.split('-')
    parts = [p.strip() for p in parts]
    if len(parts) == 1:
        return [ int(parts[0].split(' ')[0].strip()) ]
    else:
        return [ int(parts[0]), int(parts[1]) ]

In [15]:
data = []
for row in values:
    name = row[0]
    fname = "".join(x for x in name if x.isalnum())
    item = { 
        'name': name,
        'releaseDate': parse_rdate(row[3]),
        'type': row[2],
        'thumbnail': f"/images/{fname}.jpg",
        'params': {}
    }
    date = parse_date(row[1])
    if date is None:
        continue
    if len(date) == 1:
        item['year'] = date[0]
    else:
        item['startYear'] = date[0]
        item['endYear'] = date[1]
    data.append(item)